In [68]:
# https://www.kaggle.com/code/hojjatk/read-mnist-dataset/notebook

#
# This is a sample Notebook to demonstrate how to read "MNIST Dataset"
#
import numpy as np # linear algebra
import struct
from array import array
from os.path  import join

#
# MNIST Data Loader Class
#
class MnistDataloader(object):
    def __init__(self, training_images_filepath,training_labels_filepath,
                 test_images_filepath, test_labels_filepath):
        self.training_images_filepath = training_images_filepath
        self.training_labels_filepath = training_labels_filepath
        self.test_images_filepath = test_images_filepath
        self.test_labels_filepath = test_labels_filepath
    
    def read_images_labels(self, images_filepath, labels_filepath):        
        labels = []
        with open(labels_filepath, 'rb') as file:
            magic, size = struct.unpack(">II", file.read(8))
            if magic != 2049:
                raise ValueError('Magic number mismatch, expected 2049, got {}'.format(magic))
            labels = array("B", file.read())        
        
        with open(images_filepath, 'rb') as file:
            magic, size, rows, cols = struct.unpack(">IIII", file.read(16))
            if magic != 2051:
                raise ValueError('Magic number mismatch, expected 2051, got {}'.format(magic))
            image_data = array("B", file.read())        
        images = []
        for i in range(size):
            images.append([0] * rows * cols)
        for i in range(size):
            img = np.array(image_data[i * rows * cols:(i + 1) * rows * cols])
            img = img.reshape(28, 28)
            images[i][:] = img            
        
        return images, labels
            
    def load_data(self):
        x_train, y_train = self.read_images_labels(self.training_images_filepath, self.training_labels_filepath)
        x_test, y_test = self.read_images_labels(self.test_images_filepath, self.test_labels_filepath)
        return (x_train, y_train),(x_test, y_test)    

In [69]:
# # https://www.kaggle.com/code/hojjatk/read-mnist-dataset/notebook

# #
# # Verify Reading Dataset via MnistDataloader class
# #
# %matplotlib inline
# import random
# import matplotlib.pyplot as plt
# import pickle

# #
# # Set file paths based on added MNIST Datasets
# #
# input_path = 'dataset/'
# training_images_filepath = join(input_path, 'train-images-idx3-ubyte/train-images-idx3-ubyte')
# training_labels_filepath = join(input_path, 'train-labels-idx1-ubyte/train-labels-idx1-ubyte')
# test_images_filepath = join(input_path, 't10k-images-idx3-ubyte/t10k-images-idx3-ubyte')
# test_labels_filepath = join(input_path, 't10k-labels-idx1-ubyte/t10k-labels-idx1-ubyte')

# #
# # Helper function to show a list of images with their relating titles
# #
# def show_images(images, title_texts):
#     cols = 5
#     rows = int(len(images)/cols) + 1
#     plt.figure(figsize=(30,20))
#     index = 1    
#     for x in zip(images, title_texts):        
#         image = x[0]        
#         title_text = x[1]
#         plt.subplot(rows, cols, index)        
#         plt.imshow(image, cmap=plt.cm.gray)
#         if (title_text != ''):
#             plt.title(title_text, fontsize = 15);        
#         index += 1

# #
# # Load MINST dataset
# #
# mnist_dataloader = MnistDataloader(training_images_filepath, training_labels_filepath, test_images_filepath, test_labels_filepath)
# (data_train, label_train), (data_test, label_test) = mnist_dataloader.load_data()

# with open("dataset/pickled/data_train.pickle", "wb") as outfile:
#     pickle.dump(data_train, outfile)
# with open("dataset/pickled/label_train.pickle", "wb") as outfile:
#     pickle.dump(label_train, outfile)
# with open("dataset/pickled/data_test.pickle", "wb") as outfile:
#     pickle.dump(data_test, outfile)
# with open("dataset/pickled/label_test.pickle", "wb") as outfile:
#     pickle.dump(label_test, outfile)

# first_10_data_train = []
# first_10_label_train = []
# for i in range(0, 10):
#     first_10_data_train.append(data_train[i])
#     first_10_label_train.append(label_train[i])

# with open("dataset/pickled/first_10_data_train.pickle", "wb") as outfile:
#     pickle.dump(first_10_data_train, outfile)
# with open("dataset/pickled/first_10_label_train.pickle", "wb") as outfile:
#     pickle.dump(first_10_label_train, outfile)

# #
# # Show some random training and test images 
# #
# images_2_show = []
# titles_2_show = []
# for i in range(0, 10):
#    images_2_show.append(first_10_data_train[i])
#    titles_2_show.append('training image [' + str(i) + '] = ' + str(first_10_label_train[i]))    

# #for i in range(0, 5):
# #   r = random.randint(1, 10000)
# #   images_2_show.append(data_test[r])        
# #   titles_2_show.append('test image [' + str(r) + '] = ' + str(label_test[r]))    

# show_images(images_2_show, titles_2_show)


In [70]:
import random as random

class Layer:
    def __init__(self, currentLayerLen):
        self.activationVector = np.array([random.uniform(0, 1) for i in range(currentLayerLen)])
        self.biasVector = np.array([random.uniform(0, 1) for i in range(currentLayerLen)])
        self.zVector = np.zeros(shape = (currentLayerLen, 1))
        self.errorVector = np.zeros(shape = (currentLayerLen, 1))
        self.size = currentLayerLen

    def cout(self):
        print("Activations: ")
        print(self.activationVector)
        print("Biases: ")
        print(self.biasVector)
        print("Z Vector: ")
        print(self.zVector)
        print("Error: ")
        print(self.errorVector)

    def coutBase(self):
        print("Activations: ")
        print(self.activationVector)
        print("Biases: ")
        print(self.biasVector)

In [71]:
class weightMatrix:
    def __init__(self, prevLayerLen, nextLayerLen):
        self.matrix = np.random.rand(nextLayerLen, prevLayerLen)

    def cout(self):
        print(self.matrix)

In [72]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoidDeriv(x):
    return np.exp(-x) / (1 + np.exp(-x))

class Network:

    def __init__(self, start, first, second, end):
        startLayer = Layer(start)
        firstLayer = Layer(first)
        secondLayer = Layer(second)
        endLayer = Layer(end)

        self.Layers = np.array([startLayer, firstLayer, secondLayer, endLayer])

        firstMatrix = weightMatrix(start, first)
        secondMatrix = weightMatrix(first, second)
        endMatrix = weightMatrix(second, end)

        # Indexed with the layer before the matrix
        self.Matrices = np.array([firstMatrix, secondMatrix, endMatrix])

    def calculateZVector(self, layerIdx):
        # self.Layers[layerIdx - 1] is previous Layer
        # currLayer = self.Layers[layerIdx] is current Layer
        # self.Matrices[layerIdx - 1].matrix is weight matrix related to this calculation
        # self.Layers[layerIdx - 1].activationVector is previous activation
        # self.Layers[layerIdx].biasVector is current Bias

        # Weight matrix * previous activation vector
        self.Layers[layerIdx].zVector = np.dot(self.Matrices[layerIdx - 1].matrix, self.Layers[layerIdx - 1].activationVector)
        # += current bias vector
        self.Layers[layerIdx].zVector += self.Layers[layerIdx].biasVector

    def forwardPropagationStep(self, layerIdx):
        # currLayer = self.Layers[layerIdx] is current Layer

        self.calculateZVector(layerIdx)

        for i in range(self.Layers[layerIdx].size):
            self.Layers[layerIdx].activationVector[i] = sigmoid(self.Layers[layerIdx].zVector[i])

    # Assumes that data is between 0 and 255 value
    def setStartLayerActivations(self, dataset):
        # self.Layers[0] is start Layer

        if len(dataset) * len(dataset[0]) != self.Layers[0].size:
            print("There is a mismatch between the size of the input data and the start layer!")
            print("Size of dataset is: " + str(len(dataset) * len(dataset[0])))
            print("Size of first layer is: " + str(self.Layers[0].size))

        print("1) Set the activations of the first layer")

        layerIdx = 0
        for row in range(0, len(dataset)):
            for col in range(0, len(dataset[0])):
                self.Layers[0].activationVector[layerIdx] = dataset[row][col] / 255
                layerIdx += 1

    def fullForwardPropagation(self, target):
        print("2) Feedforward: Compute all activations for all layers")
        self.forwardPropagationStep(1)
        self.forwardPropagationStep(2)
        self.forwardPropagationStep(3)

        print("Target is: " + str(target))
        print("Cost is: " + str(self.cost(target)))
    
    def cost(self, target):
        # self.Layers[self.Layers.size - 1] is end Layer

        sum = 0

        for i in range(self.Layers[self.Layers.size - 1].size):
            if (i == target):
                sum += pow(self.Layers[self.Layers.size - 1].activationVector[i] - 1.0, 2)
            else:
                sum += pow(self.Layers[self.Layers.size - 1].activationVector[i], 2)
        return sum / (2 * self.Layers[self.Layers.size - 1].size)
    
    def fullBackwardPropagation(self, target):
        print("3) Output Error in last layer")
        self.calculateErrorInLastLayerForTarget(target)

        print("4) Backpropagate error: calculate error for all layers")
        self.calculateErrorFromNextLayerError(2)
        self.calculateErrorFromNextLayerError(1)
    
    # The cost function is hard coded
    def calculateErrorInLastLayerForTarget(self, target):
        # self.Layers[self.Layers.size - 1] is end Layer

        CGradient = self.Layers[self.Layers.size - 1].activationVector
        # The desired value for target is 1
        # This subtracts from the activations[target] as well !
        # The change of the activations shouldnt matter
        # It is not used later in the algo for the training example
        CGradient[target] -= 1.0

        # Apply sigmoid' to endLayer.zVector in place
        for i in range(self.Layers[self.Layers.size - 1].size):
            self.Layers[self.Layers.size - 1].zVector[i] = sigmoidDeriv(self.Layers[self.Layers.size - 1].zVector[i])

        # sigmoid' was applied to endLayer.zVector in place
        self.Layers[self.Layers.size - 1].errorVector = np.multiply(CGradient, self.Layers[self.Layers.size - 1].zVector)

    # Assumes error in next layer is up to date
    def calculateErrorFromNextLayerError(self, layerIdx):
        # self.Layers[layerIdx] is current Layer
        # self.Layers[layerIdx + 1] is next Layer

        # self.Matrices[layerIdx].matrix.transpose() is the transposed weight matrix
        # np.dot(transposedWeightMatrix, self.Layers[layerIdx + 1].errorVector) is transposed weight matrix * next layer error

        # Apply sigmoid' to currLayer.zVector in place
        for i in range(self.Layers[layerIdx].size):
            self.Layers[layerIdx].zVector[i] = sigmoidDeriv(self.Layers[layerIdx].zVector[i])

        # sigmoid' was applied to self.Layers[self.Layers.size - 1].zVector in place
        # error = (transposed weight matrix * next layer error) o sigmoid'(z)
        # Where o is index by index multiplication
        self.Layers[layerIdx].errorVector = np.multiply(np.dot(self.Matrices[layerIdx].matrix.transpose(), self.Layers[layerIdx + 1].errorVector), self.Layers[layerIdx].zVector)

    # All of this would be MUCH easier if W and B were stored as matrices and vectors
    def adjustBasedOnGradientDescentForCurrentExample(self, learningRate, numberInBatch):
        print("5) Gradient Descent")

        # npm stands for n per m
        # where: n: learning rate
        #        m: number of train examples in batch
        npm = learningRate / numberInBatch

        # Adjust biases
        print(self.Layers.size)
        for layerIdx in range(1, self.Layers.size):
            # self.Layers[layerIdx] is current Layer

            self.Layers[layerIdx].biasVector -= npm * self.Layers[layerIdx].errorVector

        # # Adjust weights
        for weightMatrixIdx in range(self.Matrices.size):
            # self.Matrices[weightMatrixIdx].matrix is current weight matrix
            # self.Layers[weightMatrixIdx + 1] is current Layer
            # self.Layers[weightMatrixIdx] is previous Layer

            # For matrix mupltiplications the vectors need to be 2D
            # This is how we make them 2D
            # np.array([self.Layers[weightMatrixIdx].activationVector]) is previous activations
            # np.array([self.Layers[weightMatrixIdx + 1].errorVector]) is current error
            # Temp variable to make code more understandable
            
            # weight matrix -= (current Error)T * previous activation
            self.Matrices[weightMatrixIdx].matrix -= npm * np.dot(np.array([self.Layers[weightMatrixIdx + 1].errorVector]).transpose(), np.array([self.Layers[weightMatrixIdx].activationVector]))

    def trainBatch(self, data, labels, learningRate):
        # Check if there is a length mismatch
        if (len(data) != len(labels)):
            print("There is a mismatch between the length of the data and lables")
            print("Length of data is: " + str(data.size))
            print("Length of labels are: " + str(labels.size))

        numberInBatch = len(data)

        for idx in range(numberInBatch):
            # Steps of one training 
            self.setStartLayerActivations(data[idx])
            self.fullForwardPropagation(labels[idx])
            self.fullBackwardPropagation(labels[idx])
            self.adjustBasedOnGradientDescentForCurrentExample(learningRate, numberInBatch)

    def coutBase(self):
        for i in range(self.Matrices.size):
            print("Layer: ", i)
            self.Layers[i].coutBase()
            print()
            self.Matrices[i].cout()
            print()

        print("Layer: ", self.Layers.size - 1)
        self.Layers[self.Layers.size - 1].coutBase()

    def cout(self):
        for i in range(self.Matrices.size):
            print("Layer: ", i)
            self.Layers[i].cout()
            print()
            self.Matrices[i].cout()
            print()

        print("Layer: ", self.Layers.size - 1)
        self.Layers[self.Layers.size - 1].cout()

In [73]:
# MNIST Dataset: 28 x 28 = 784
import pickle

with open("network.pickle", "rb") as infile:
   net = pickle.load(infile)

#net = Network(4, 3, 3, 2)
#net = Network(784, 16, 16, 10)

# Network is loaded into net

with open("dataset/pickled/first_10_data_train.pickle", "rb") as infile:
    data_batch = pickle.load(infile)
with open("dataset/pickled/first_10_label_train.pickle", "rb") as infile:
    label_batch = pickle.load(infile)

# Training batch (first 10 elements) is loaded into data and label _batch-es

net.trainBatch(data_batch, label_batch, 10)

with open("network.pickle", "wb") as outfile:
    pickle.dump(net, outfile)

1) Set the activations of the first layer
2) Feedforward: Compute all activations for all layers
Target is: 5
Cost is: 0.4497302861843407
3) Output Error in last layer
4) Backpropagate error: calculate error for all layers
5) Gradient Descent
4
1) Set the activations of the first layer
2) Feedforward: Compute all activations for all layers
Target is: 0
Cost is: 0.44966049191170043
3) Output Error in last layer
4) Backpropagate error: calculate error for all layers
5) Gradient Descent
4
1) Set the activations of the first layer
2) Feedforward: Compute all activations for all layers
Target is: 4
Cost is: 0.4496302277562741
3) Output Error in last layer
4) Backpropagate error: calculate error for all layers
5) Gradient Descent
4
1) Set the activations of the first layer
2) Feedforward: Compute all activations for all layers
Target is: 1
Cost is: 0.44977914166102806
3) Output Error in last layer
4) Backpropagate error: calculate error for all layers
5) Gradient Descent
4
1) Set the activat